In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dlp-jan-2025-nppe-3/sample_submission.csv
/kaggle/input/dlp-jan-2025-nppe-3/submission.py
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00159.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00056.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00017.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00124.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00140.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00068.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00019.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00266.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00236.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00148.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00152.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00226.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00008.png
/kaggle/input/dlp-jan-2025-nppe-3/archive/val/gt/gt_00216.png
/kaggle/input/dlp-jan-2025-n

In [17]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.amp import autocast, GradScaler

# Dataset Classes
class LowLightDataset(Dataset):
    def __init__(self, noisy_dir, gt_dir):
        self.noisy_dir = noisy_dir
        self.gt_dir = gt_dir
        self.noisy_files = sorted(os.listdir(noisy_dir))
        self.gt_files = sorted(os.listdir(gt_dir))
        assert len(self.noisy_files) == len(self.gt_files), "Mismatch in file counts"

    def __len__(self):
        return len(self.noisy_files)

    def __getitem__(self, idx):
        noisy_path = os.path.join(self.noisy_dir, self.noisy_files[idx])
        gt_path = os.path.join(self.gt_dir, self.gt_files[idx])
        
        noisy_img = np.array(Image.open(noisy_path)) / 255.0
        gt_img = np.array(Image.open(gt_path)) / 255.0
        
        noisy_img = torch.FloatTensor(noisy_img).permute(2, 0, 1)
        gt_img = torch.FloatTensor(gt_img).permute(2, 0, 1)
        
        return noisy_img, gt_img

class TestDataset(Dataset):
    def __init__(self, test_dir):
        self.test_dir = test_dir
        self.test_files = sorted(os.listdir(test_dir))

    def __len__(self):
        return len(self.test_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.test_dir, self.test_files[idx])
        img = np.array(Image.open(img_path)) / 255.0
        img = torch.FloatTensor(img).permute(2, 0, 1)
        return img, self.test_files[idx]
        

In [18]:
# Main Execution
#base_path = "path_to_dataset"  # Replace with your dataset path
train_noisy_path = os.path.join(base_path, "train/train")
train_gt_path = os.path.join(base_path, "train/gt")
val_noisy_path = os.path.join(base_path, "val/val")
val_gt_path = os.path.join(base_path, "val/gt")
test_dir = os.path.join(base_path, "test")
output_folder = "test_outputs"
submission_csv = "submission.csv"

In [19]:
# Generator
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.entry = nn.Conv2d(3, 64, 3, padding=1)
        self.body = nn.Sequential(
            *[nn.Sequential(
                nn.Conv2d(64, 64, 3, padding=1),
                nn.ReLU()
            ) for _ in range(5)]
        )
        self.upscale = nn.Sequential(
            nn.Conv2d(64, 64 * 16, 3, padding=1),
            nn.PixelShuffle(4),
            nn.Conv2d(64, 1, 3, padding=1)
        )
    
    def forward(self, x):
        x = self.entry(x)
        res = self.body(x)
        return self.upscale(res + x)

# Discriminator (remove Sigmoid)
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 64, 3, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 3, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 3, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.Conv2d(256, 512, 3, stride=2, padding=1), nn.LeakyReLU(0.2),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(512, 256), nn.LeakyReLU(0.2),
            nn.Linear(256, 1)  # Output logits, no Sigmoid
        )
    
    def forward(self, x):
        return self.model(x)



In [29]:
def train_sgan(train_loader, val_loader, epochs=50):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    gen = Generator().to(device)
    disc = Discriminator().to(device)
    
    print("No EDSR checkpoints found—training SRGAN from scratch")  # Since confirmed
    
    g_opt = torch.optim.Adam(gen.parameters(), lr=1e-4)
    d_opt = torch.optim.Adam(disc.parameters(), lr=1e-4)
    mse_loss = nn.MSELoss()
    bce_logits_loss = nn.BCEWithLogitsLoss()
    scaler = GradScaler('cuda')

    for epoch in range(epochs):
        gen.train()
        disc.train()
        g_loss_total, d_loss_total = 0, 0
        for noisy, gt in train_loader:
            noisy, gt = noisy.to(device), gt.to(device)
            gt_gray = torch.mean(gt, dim=1, keepdim=True)
            
            # Train Discriminator
            d_opt.zero_grad()
            with autocast('cuda'):
                real_out = disc(gt_gray)
                fake_img = gen(noisy)
                fake_out = disc(fake_img.detach())
                d_loss = bce_logits_loss(real_out, torch.ones_like(real_out)) + \
                         bce_logits_loss(fake_out, torch.zeros_like(fake_out))
            scaler.scale(d_loss).backward()
            scaler.step(d_opt)
            scaler.update()
            d_loss_total += d_loss.item()
            
            # Train Generator
            g_opt.zero_grad()
            with autocast('cuda'):
                fake_out = disc(fake_img)
                g_mse = mse_loss(fake_img, gt_gray)
                g_adv = bce_logits_loss(fake_out, torch.ones_like(fake_out))
                g_loss = 0.95 * g_mse + 0.05 * g_adv  # Adjusted weights
            scaler.scale(g_loss).backward()
            scaler.step(g_opt)
            scaler.update()
            g_loss_total += g_mse.item()
        
        avg_g_loss = g_loss_total / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, G Loss (MSE): {avg_g_loss:.6f}")
        
        gen.eval()
        val_loss = 0
        with torch.no_grad():
            for noisy, gt in val_loader:
                noisy, gt = noisy.to(device), gt.to(device)
                with autocast('cuda'):
                    output = gen(noisy)
                    gt_gray = torch.mean(gt, dim=1, keepdim=True)
                    loss = mse_loss(output, gt_gray)
                val_loss += loss.item()
        val_mse = val_loss / len(val_loader)
        val_psnr = 10 * torch.log10(1 / torch.tensor(val_mse)).item()
        print(f"Validation Loss: {val_mse:.6f}, PSNR: {val_psnr:.2f} dB")
        
        if (epoch + 1) % 10 == 0:
            torch.save(gen.state_dict(), f"srgan_gen_epoch_{epoch+1}.pth")
    
    torch.save(gen.state_dict(), "srgan_gen_final.pth")
    return gen

In [30]:
# Submission Function
def generate_submission(model, test_dir, output_folder, submission_csv):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device).eval()
    os.makedirs(output_folder, exist_ok=True)
    test_dataset = TestDataset(test_dir)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    with torch.no_grad():
        for imgs, filenames in test_loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            output = outputs.squeeze(0).cpu().numpy()[0]
            output = (output * 255).clip(0, 255).astype(np.uint8)
            filename = filenames[0]
            output_path = os.path.join(output_folder, filename)
            Image.fromarray(output).save(output_path)
    data_rows = []
    for filename in os.listdir(output_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(output_folder, filename)
            image = Image.open(image_path).convert('L')
            image_array = np.array(image).flatten()[::8]
            image_id = filename.split('.')[0].replace('test_', 'gt_')
            data_rows.append([image_id, *image_array])
    column_names = ['ID'] + [f'pixel_{i}' for i in range(len(data_rows[0]) - 1)]
    df = pd.DataFrame(data_rows, columns=column_names)
    df.to_csv(submission_csv, index=False)
    print(f'Successfully saved to {submission_csv}')

In [31]:
# Main Execution
base_path = "/kaggle/input/dlp-jan-2025-nppe-3/archive"  
train_noisy_path = os.path.join(base_path, "train/train")
train_gt_path = os.path.join(base_path, "train/gt")
val_noisy_path = os.path.join(base_path, "val/val")
val_gt_path = os.path.join(base_path, "val/gt")
test_dir = os.path.join(base_path, "test")
output_folder = "test_outputs"
submission_csv = "submission.csv"

In [32]:
train_dataset = LowLightDataset(train_noisy_path, train_gt_path)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataset = LowLightDataset(val_noisy_path, val_gt_path)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
    

In [33]:
model = train_sgan(train_loader, val_loader, epochs=50) #50

No EDSR checkpoints found—training SRGAN from scratch
Epoch 1/50, G Loss (MSE): 0.003712
Validation Loss: 0.000415, PSNR: 33.82 dB
Epoch 2/50, G Loss (MSE): 0.000639
Validation Loss: 0.000308, PSNR: 35.11 dB
Epoch 3/50, G Loss (MSE): 0.000459
Validation Loss: 0.000245, PSNR: 36.11 dB
Epoch 4/50, G Loss (MSE): 0.000390
Validation Loss: 0.000224, PSNR: 36.51 dB
Epoch 5/50, G Loss (MSE): 0.000371
Validation Loss: 0.000235, PSNR: 36.28 dB
Epoch 6/50, G Loss (MSE): 0.000391
Validation Loss: 0.000195, PSNR: 37.10 dB
Epoch 7/50, G Loss (MSE): 0.000321
Validation Loss: 0.000222, PSNR: 36.53 dB
Epoch 8/50, G Loss (MSE): 0.000337
Validation Loss: 0.000195, PSNR: 37.09 dB
Epoch 9/50, G Loss (MSE): 0.000321
Validation Loss: 0.000213, PSNR: 36.71 dB
Epoch 10/50, G Loss (MSE): 0.000301
Validation Loss: 0.000196, PSNR: 37.09 dB
Epoch 11/50, G Loss (MSE): 0.000304
Validation Loss: 0.000182, PSNR: 37.39 dB
Epoch 12/50, G Loss (MSE): 0.000289
Validation Loss: 0.000171, PSNR: 37.67 dB
Epoch 13/50, G Loss

KeyboardInterrupt: 

In [34]:
# Training SRGAN
def train_sgan(train_loader, val_loader, epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    gen = Generator().to(device)
    disc = Discriminator().to(device)
    
    # Load Epoch 20 checkpoint
    checkpoint_path = "srgan_gen_epoch_20.pth"
    try:
        gen.load_state_dict(torch.load(checkpoint_path))
        print(f"Loaded checkpoint from {checkpoint_path}, continuing from Epoch 20")
    except FileNotFoundError:
        print(f"Checkpoint {checkpoint_path} not found—training from scratch")
    
    g_opt = torch.optim.Adam(gen.parameters(), lr=1e-4)
    d_opt = torch.optim.Adam(disc.parameters(), lr=1e-4)
    g_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(g_opt, 'min', patience=5, factor=0.5)
    d_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(d_opt, 'min', patience=5, factor=0.5)
    mse_loss = nn.MSELoss()
    bce_logits_loss = nn.BCEWithLogitsLoss()
    scaler = GradScaler('cuda')

    for epoch in range(20, epochs):  # Start from 21 since loaded Epoch 20
        gen.train()
        disc.train()
        g_loss_total, d_loss_total = 0, 0
        for noisy, gt in train_loader:
            noisy, gt = noisy.to(device), gt.to(device)
            gt_gray = torch.mean(gt, dim=1, keepdim=True)
            
            # Train Discriminator
            d_opt.zero_grad()
            with autocast('cuda'):
                real_out = disc(gt_gray)
                fake_img = gen(noisy)
                fake_out = disc(fake_img.detach())
                d_loss = bce_logits_loss(real_out, torch.ones_like(real_out)) + \
                         bce_logits_loss(fake_out, torch.zeros_like(fake_out))
            scaler.scale(d_loss).backward()
            scaler.step(d_opt)
            scaler.update()
            d_loss_total += d_loss.item()
            
            # Train Generator
            g_opt.zero_grad()
            with autocast('cuda'):
                fake_out = disc(fake_img)
                g_mse = mse_loss(fake_img, gt_gray)
                g_adv = bce_logits_loss(fake_out, torch.ones_like(fake_out))
                g_loss = 0.95 * g_mse + 0.05 * g_adv
            scaler.scale(g_loss).backward()
            scaler.step(g_opt)
            scaler.update()
            g_loss_total += g_mse.item()
        
        avg_g_loss = g_loss_total / len(train_loader)
        avg_d_loss = d_loss_total / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, G Loss (MSE): {avg_g_loss:.6f}, D Loss: {avg_d_loss:.6f}")
        
        gen.eval()
        val_loss = 0
        with torch.no_grad():
            for noisy, gt in val_loader:
                noisy, gt = noisy.to(device), gt.to(device)
                with autocast('cuda'):
                    output = gen(noisy)
                    gt_gray = torch.mean(gt, dim=1, keepdim=True)
                    loss = mse_loss(output, gt_gray)
                val_loss += loss.item()
        val_mse = val_loss / len(val_loader)
        val_psnr = 10 * torch.log10(1 / torch.tensor(val_mse)).item()
        print(f"Validation Loss: {val_mse:.6f}, PSNR: {val_psnr:.2f} dB")
        
        g_scheduler.step(val_mse)
        d_scheduler.step(avg_d_loss)
        
        if (epoch + 1) % 10 == 0:
            torch.save(gen.state_dict(), f"srgan_gen_epoch_{epoch+1}.pth")
            print(f"Saved checkpoint: srgan_gen_epoch_{epoch+1}.pth")
    
    torch.save(gen.state_dict(), "srgan_gen_final.pth")
    return gen

In [ ]:
model = train_sgan(train_loader, val_loader, epochs=100)

In [37]:
model.load_state_dict(torch.load("srgan_gen_epoch_20.pth"))

<ipython-input-37-577406f7fa57>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("srgan_gen_epoch_20.pth"))


<All keys matched successfully>

In [38]:
generate_submission(model, test_dir, output_folder, submission_csv)

Successfully saved to submission.csv


In [39]:
# Training SRGAN
def train_sgan(train_loader, val_loader, epochs=100):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    gen = Generator().to(device)
    disc = Discriminator().to(device)
    
    # Load Epoch 20 checkpoint
    checkpoint_path = "srgan_gen_epoch_20.pth"
    try:
        gen.load_state_dict(torch.load(checkpoint_path))
        print(f"Loaded checkpoint from {checkpoint_path}, continuing from Epoch 20")
    except FileNotFoundError:
        print(f"Checkpoint {checkpoint_path} not found—training from scratch")
    
    g_opt = torch.optim.Adam(gen.parameters(), lr=5e-5)  # Lower LR
    d_opt = torch.optim.Adam(disc.parameters(), lr=5e-5)  # Lower LR
    g_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(g_opt, 'min', patience=5, factor=0.5)
    d_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(d_opt, 'min', patience=5, factor=0.5)
    mse_loss = nn.MSELoss()
    bce_logits_loss = nn.BCEWithLogitsLoss()
    scaler = GradScaler('cuda')

    for epoch in range(20, epochs):
        gen.train()
        disc.train()
        g_loss_total, d_loss_total = 0, 0
        for noisy, gt in train_loader:
            noisy, gt = noisy.to(device), gt.to(device)
            gt_gray = torch.mean(gt, dim=1, keepdim=True)
            
            # Train Discriminator
            d_opt.zero_grad()
            with autocast('cuda'):
                real_out = disc(gt_gray)
                fake_img = gen(noisy)
                fake_out = disc(fake_img.detach())
                d_loss = bce_logits_loss(real_out, torch.ones_like(real_out)) + \
                         bce_logits_loss(fake_out, torch.zeros_like(fake_out))
            scaler.scale(d_loss).backward()
            scaler.step(d_opt)
            scaler.update()
            d_loss_total += d_loss.item()
            
            # Train Generator
            g_opt.zero_grad()
            with autocast('cuda'):
                fake_out = disc(fake_img)
                g_mse = mse_loss(fake_img, gt_gray)
                g_adv = bce_logits_loss(fake_out, torch.ones_like(fake_out))
                g_loss = 0.9 * g_mse + 0.1 * g_adv  # Adjusted weights
            scaler.scale(g_loss).backward()
            scaler.step(g_opt)
            scaler.update()
            g_loss_total += g_mse.item()
        
        avg_g_loss = g_loss_total / len(train_loader)
        avg_d_loss = d_loss_total / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, G Loss (MSE): {avg_g_loss:.6f}, D Loss: {avg_d_loss:.6f}")
        
        gen.eval()
        val_loss = 0
        with torch.no_grad():
            for noisy, gt in val_loader:
                noisy, gt = noisy.to(device), gt.to(device)
                with autocast('cuda'):
                    output = gen(noisy)
                    gt_gray = torch.mean(gt, dim=1, keepdim=True)
                    loss = mse_loss(output, gt_gray)
                val_loss += loss.item()
        val_mse = val_loss / len(val_loader)
        val_psnr = 10 * torch.log10(1 / torch.tensor(val_mse)).item()
        print(f"Validation Loss: {val_mse:.6f}, PSNR: {val_psnr:.2f} dB")
        
        g_scheduler.step(val_mse)
        d_scheduler.step(avg_d_loss)
        
        if (epoch + 1) % 10 == 0:
            torch.save(gen.state_dict(), f"srgan_gen_epoch_{epoch+1}.pth")
            print(f"Saved checkpoint: srgan_gen_epoch_{epoch+1}.pth")
    
    torch.save(gen.state_dict(), "srgan_gen_final.pth")
    return gen

In [ ]:
model = train_sgan(train_loader, val_loader, epochs=100)

In [ ]:
generate_submission(model, test_dir, output_folder, submission_csv)